# Programa Ingenias+ Data Science

## Objetivo del notebook

El objetivo de este notebook es aplicar y comparar técnicas de aprendizaje no supervisado, tanto de clustering como de reducción de dimensionalidad, para identificar patrones y estructuras subyacentes en los datos.

Las columnas de características que se utilizarán en el notebook se detallan a continuación.


## Importación de archivos y librerías

In [14]:
# Imports de librerías y funciones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from utils.codificaciones import *
from utils.funcions import *

In [4]:
# Importar el csv
friesgo_raw = pd.read_csv('datasets/friesgo.csv')

## Preprocesado

### Identificación de variables

In [15]:
# Separar variables categóricas y numéricas
variables_categoricas = ['cod_provincia','tamanio_aglomerado','sexo','condicion_actividad','ansiedad_depresion'
                           ,'nivel_actividad_fisica','condicion_fumador','consumo_sal'
                           ,'colesterol_alto','freq_cons_alc','es_diabetico','mantiene_ha_alta']
variables_numericas = ['edad','media_fv_diaria','imc_numerico']

In [6]:
# Selección de columnas
friesgo = friesgo_raw[['cod_provincia','tamanio_aglomerado','sexo','condicion_actividad','ansiedad_depresion',
           'nivel_actividad_fisica','condicion_fumador','consumo_sal','colesterol_alto',
           'freq_cons_alc','es_diabetico','mantiene_ha_alta', 'edad','media_fv_diaria',
           'imc_numerico']]

In [7]:
# Copiar el dataframe
friesgo_cod = friesgo.copy()

### Evaluación y procesamiento de nulos

In [10]:
calcular_nulos_y_porcentaje(friesgo_cod)

,Nulos,Porcentaje
cod_provincia,0,0.0%
tamanio_aglomerado,0,0.0%
sexo,0,0.0%
condicion_actividad,0,0.0%
ansiedad_depresion,0,0.0%
nivel_actividad_fisica,226,0.84%
condicion_fumador,0,0.0%
consumo_sal,0,0.0%
colesterol_alto,0,0.0%
freq_cons_alc,0,0.0%


In [11]:
# Eliminación de nulos
friesgo_cod = friesgo_cod.dropna()

In [12]:
calcular_nulos_y_porcentaje(friesgo_cod)

,Nulos,Porcentaje
cod_provincia,0,0.0%
tamanio_aglomerado,0,0.0%
sexo,0,0.0%
condicion_actividad,0,0.0%
ansiedad_depresion,0,0.0%
nivel_actividad_fisica,0,0.0%
condicion_fumador,0,0.0%
consumo_sal,0,0.0%
colesterol_alto,0,0.0%
freq_cons_alc,0,0.0%


### Codificación de columnas

In [13]:
# Visualización de filas y columnas
friesgo_cod.shape

(25539, 15)

In [16]:
# Codificación de columnas
le = LabelEncoder()
for columnas in variables_categoricas:
    friesgo_cod[columnas] = le.fit_transform(friesgo_cod[columnas])

In [17]:
# Evaluar que no haya pérdida de filas y/o columnas por la codificación
friesgo_cod.shape

(25539, 15)

## Análisis de modelos

### Modelo 1: K-Means

### Modelo 2: DBSCAN

### Modelo 3: